<a href="https://colab.research.google.com/github/Elvira-Zainulina/ml_MRI_age_prediction/blob/master/Data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
! pip install torchio

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
import numpy as np
import pandas as pd
import os
from ml_utils.uploading import unzip_imgs
from ml_utils.uploading import upload_raw_data
import pickle
from ml_utils.data import S500MRI_Dataset
from ml_utils.preprocessing import get_hist_landmarks

If you use TorchIO for your research, please cite the following paper:

Pérez-García et al., TorchIO: a Python library for efficient loading,
preprocessing, augmentation and patch-based sampling of medical images
in deep learning
(https://arxiv.org/abs/2003.04696)


In [0]:
path = r"drive/My Drive/Colab Notebooks/ml-project/check"

archive_path = os.path.join(path, 'tmp')
images_path = os.path.join(path, 'images')
table_path = os.path.join(path, 'data', 'table_with_ages.csv')

In [3]:
unzip_imgs(archive_path, images_path)

Unzipped 185442_3T_Structural_preproc.zip
Unzipped 181131_3T_Structural_preproc.zip
Unzipped 181232_3T_Structural_preproc.zip
Unzipped 182739_3T_Structural_preproc.zip
Unzipped 182840_3T_Structural_preproc.zip
Unzipped 183034_3T_Structural_preproc.zip
Unzipped 185139_3T_Structural_preproc.zip


In [0]:
train_names = np.load(os.path.join(path, 'data', 'train_names.npy'))
test_names = np.load(os.path.join(path, 'data', 'test_names.npy'))
val_names = np.load(os.path.join(path, 'data', 'val_names.npy'))

In [0]:
train_data = upload_raw_data(images_path, table_path, train_names)
val_data = upload_raw_data(images_path, table_path, val_names)
test_data = upload_raw_data(images_path, table_path, test_names)

In [0]:
with open(os.path.join(path, 'data', 'train_data.pickle'), 'wb') as f:
    pickle.dump(train_data, f)

with open(os.path.join(path, 'data', 'val_data.pickle'), 'wb') as f:
    pickle.dump(val_data, f)

with open(os.path.join(path, 'data', 'test_data.pickle'), 'wb') as f:
    pickle.dump(test_data, f)

In [0]:
with open(os.path.join(path, 'data', 'train_data.pickle'), 'rb') as f:
    train_data = pickle.load(f)

with open(os.path.join(path, 'data', 'val_data.pickle'), 'rb') as f:
    val_data = pickle.load(f)

with open(os.path.join(path, 'data', 'test_data.pickle'), 'rb') as f:
    test_data = pickle.load(f)

In [9]:
landmarks_path = os.path.join(path, 'data', 'landmarks.npy')

landmarks = get_hist_landmarks(images_path, landmarks_path,
                               np.append(train_names, val_names))

100%|██████████| 5/5 [00:04<00:00,  1.22it/s]


In [0]:
from torchio.transforms import (
    ToCanonical,
    ZNormalization,
    HistogramStandardization,
)

from torchvision.transforms import Compose
from ml_utils.transforms import ResizeCrop, BoundsCrop

In [0]:
transform = Compose([
    BoundsCrop(),                            
    HistogramStandardization(landmarks_dict={'MRI': landmarks}),
    ZNormalization(masking_method=ZNormalization.mean),
    ToCanonical(),
    ResizeCrop(128)
])

In [0]:
train_dataset = S500MRI_Dataset(**train_data, transform=transform)
val_dataset = S500MRI_Dataset(**val_data, transform=transform)
test_dataset = S500MRI_Dataset(**test_data, transform=transform)

In [0]:
from ml_utils.preprocessing import process_dataset

In [17]:
_ = process_dataset(train_dataset, os.path.join(path, 'data', 'train_dataset.npy'))
_ = process_dataset(val_dataset, os.path.join(path, 'data', 'val_dataset.npy'))
_ = process_dataset(test_dataset, os.path.join(path, 'data', 'test_dataset.npy'))